# LaTeX-based Resume Generator

## Problem Statement
Creating a professional, well-formatted resume in LaTeX is difficult for most applicants, especially those unfamiliar with LaTeX syntax. Editing and reformatting an existing resume to match a new template is tedious and error-prone. Applicants often want to quickly generate a resume from raw text or update its style—without wrestling with LaTeX code.

## Solution Overview
While there are platforms like overleaf where editing LaTeX documents are way easier, we have brought something simpler. This notebook demonstrates the use of Generative AI (Google Gemini) to:

* Parse your plain text resume and extract all important details automatically.
* Instantly generate a new LaTeX resume in your chosen style, simply by uploading a sample template.
* Eliminate the need for manual LaTeX editing, making it easy to create or reformat resumes for any application.

By combining GenAI’s text understanding with LaTeX’s formatting power, applicants can generate professional resumes in any style—hassle-free.

## Install Required Packages and Tools

We install:
* **google-genai**: To access Google Gemini for parsing and generating resume content.
* **LaTeX packages**: To compile the AI-generated LaTeX code into a PDF resume.

In [1]:
!pip install -qU google-genai==1.7.0
!apt-get update
!apt-get install -y texlive-latex-base texlive-latex-extra texlive-fonts-recommended

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 2.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [73.0 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Get:11 https://r2u.

## Import Python Libraries

We import:
* **google.genai**: For communicating with the Gemini API.
* **json**: For handling structured resume data.
* **IPython.display**: For showing the final PDF and download links in the notebook.

In [2]:
import google.genai as genai
from google.genai import types
import json
from IPython.display import Markdown, display, IFrame, FileLink

## Configure Google Gemini API Key

The Gemini API key is securely loaded from Kaggle Secrets.

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

## Provide Your Resume Text

Paste your resume here in plain text format. The AI will extract all relevant information from this text, so you don’t need to worry about formatting or structure.

A sample text is given, but it does not have to be in same format. **Gemini** can handle long context, so even if you have multipage resume, it shouldn't be a problem.

In [4]:
resume_text = """
John Doe
john.doe@email.com
(555) 555-5555
linkedin.com/in/johndoe
github.com/johndoe

Objective: I’m a data scientist looking to use data to solve real-world problems. Skilled in machine learning, data analysis, and building models to make sense of big data. I want to use my skills to help organizations make data-driven decisions.

Education: M.S. in Data Science, XYZ University, 2022. B.S. in Computer Science, ABC University, 2020.

Skills: Python, R, SQL, Java, Pandas, NumPy, Scikit-learn, TensorFlow, Keras, XGBoost, Matplotlib, Seaborn, Tableau, MySQL, PostgreSQL, MongoDB, Git, Jupyter, Docker, Hadoop, AWS, Google Cloud.

Experience: Data Scientist Intern, XYZ Tech Company, 2021. Analyzed customer data to identify behavior patterns and trends. Built machine learning models to predict customer churn. Created data visualizations and dashboards using Tableau for internal teams. Data Analyst, ABC Analytics, 2020. Extracted, cleaned, and analyzed data using SQL and Python. Developed reports and visualizations to provide actionable insights for clients. Built a predictive model for sales forecasting using linear regression.

Projects: Customer Segmentation - Applied K-Means clustering to segment customers based on purchasing behavior. Sales Forecasting Model - Built a linear regression model to predict future sales with 85% accuracy. Sentiment Analysis on Social Media Data - Used NLP techniques to analyze social media posts for sentiment.

Certifications: Data Science Specialization (Coursera). Machine Learning by Andrew Ng (Coursera). AWS Certified Machine Learning - Specialty.

Interests: Kaggle competitions, Open-source contributions, Hiking, Photography, Blogging about data science.
"""

## Upload and Load Your Desired LaTeX Resume Template

Upload a sample LaTeX template that matches the style you want for your resume. The AI will use this as a reference to generate your new resume. 

### To use your own template
You can upload your LaTeX template in google drive with share to "anyone with link".
Then paste the share link below, replacing the existing one.

**NOTE**: The file name should be **sample_resume.tex**

In [5]:
!gdown --fuzzy "https://drive.google.com/file/d/18SlVKUNXechAByZwIS4VvPxJ3p2O5kj2/view?usp=drive_link"

with open('sample_resume.tex') as f:
    latex_template = f.read()

Downloading...
From: https://drive.google.com/uc?id=18SlVKUNXechAByZwIS4VvPxJ3p2O5kj2
To: /kaggle/working/sample_resume.tex
100%|██████████████████████████████████████| 7.87k/7.87k [00:00<00:00, 18.7MB/s]


## Build the Resume Parsing Prompt

We prepare a detailed prompt for Gemini, specifying the fields to extract (name, contact, education, experience, etc.) and providing an example. This helps the AI accurately convert your raw text into structured JSON data.

In [6]:
prompt = f"""
You are a precise and flexible resume parser.

Convert the following resume into structured JSON with these fields:

- name
- contact: {{email, phone, linkedin, github, website}}
- summary (if present, otherwise empty string)
- education: list of {{degree, university, year, gpa (if present)}}
- experience: list of {{title, company, location (if present), years, descriptions (list of bullet points)}}
- skills: {{technical (list), soft (list if present), languages (list if present)}}
- projects (if present): list of {{name, description, technologies}}
- other_sections (if present): dictionary where keys are section names (like 'Certifications', 'Awards', 'Publications', etc.) and values are lists of strings or key-value objects where appropriate.

Any information not fitting the core fields should go into `other_sections`.

For missing information, use empty strings or empty lists.

Here is an example:

Resume:
Jane Smith  
Email: jane@example.com  
Phone: 555-321-9876  
LinkedIn: linkedin.com/in/janesmith  
GitHub: github.com/janesmith  

Education:  
M.Sc. in Data Science, ABC University, 2021  

Experience:  
- Data Scientist at XYZ Corp (2021–2023)  
  Led model development for customer churn prediction using Python and scikit-learn.  
  Improved accuracy by 15% through hyperparameter tuning.  
  Deployed models using Docker and Flask APIs.  

Skills:  
Python, Machine Learning, Pandas, Docker, Flask  

Projects:  
- Customer Churn Predictor  
  Developed a model to predict churn with 85% accuracy using historical behavior data.  
  Technologies: Python, scikit-learn, pandas  

Certifications:  
- AWS Certified Machine Learning – Specialty  

---

Now parse the following resume:

Resume:
{resume_text}
"""


## Parse Resume Text into Structured Data

We use the **gemini-1.5-pro-latest** to convert raw text to json. MIME type json is used to get a pure json output.

In [7]:
response = client.models.generate_content(
    model = "models/gemini-1.5-pro-latest",
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.2,
        response_mime_type="application/json"
    )
)

resume_json = json.loads(response.candidates[0].content.parts[0].text)

# Display result
# display(Markdown("### Structured Resume JSON"))
# print(json.dumps(resume_json, indent=2))

## Build the LaTeX Generation Prompt

We instruct Gemini to use your structured resume data and the uploaded LaTeX template to generate a complete, ready-to-compile LaTeX document. The AI ensures the output matches your chosen style and includes all your information.


In [8]:
resume_latex_prompt = f"""
You are a LaTeX resume generator. I have given you a resume template and a resume JSON.

{json.dumps(resume_json)}

Please format it using the following LaTeX template as inspiration:

{latex_template}

Generate a complete, well-formatted LaTeX document starting with \documentclass that follows the style of the template while incorporating all the information from the JSON data. The output should be ready to compile into a PDF. 
Return only raw LaTeX code without wrapping it in triple backticks or Markdown formatting. Do not prefix the code with ```latex or anything else. Start with \documentclass....
Only include sections if they contain at least one experience, project, or education entry. Do not include empty LaTeX environments with no items.
"""

## Generate Your LaTeX Resume

Gemini creates a new LaTeX resume document, formatted according to your template.

In [9]:
latex_resume = client.models.generate_content(
    contents = resume_latex_prompt, 
    model = "models/gemini-1.5-pro-latest" ).candidates[0].content.parts[0].text

with open("generated_resume.tex", "w", encoding="utf-8") as f:
    f.write(latex_resume)
    
# display(Markdown("### Generated LaTeX Resume"))
# print(latex_resume)

## Compile the LaTeX Resume to PDF

We use LaTeX tools to compile the generated document into a PDF. This produces a polished, professional resume you can immediately use.

In [10]:
!pdflatex generated_resume.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./generated_resume.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size11.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/latexsym.sty)
(/usr/share/texlive/texmf-dist/tex/latex/preprint/fullpage.sty)
(/usr/share/texlive/texmf-dist/tex/latex/titlesec/titlesec.sty)
(/usr/share/texlive/texmf-dist/tex/latex/marvosym/marvosym.sty)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/color.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics-cfg/color.cfg)
(/usr/share/texlive/texmf-dist/tex/latex/graphics-def/pdftex.def)
(/usr/share/texlive/texmf-dist/tex/latex/graphics/dvipsnam.def))
(/usr/share/texlive/texmf-dist/tex/latex/tools/verbatim

## View and Download Your New Resume

Preview your generated PDF resume directly in the notebook, and download it for job applications or further customization.

In [11]:
display(IFrame("generated_resume.pdf", width=600, height=800))
display(FileLink(r'generated_resume.pdf'))
display(FileLink(r'generated_resume.tex'))

/kaggle/working/generated_resume.pdf

/kaggle/working/generated_resume.tex

## Conclusion

This notebook solves the problem of resume formatting for applicants by automating the process end-to-end. With GenAI, you can instantly convert raw text into a professional resume in any LaTeX style—just by uploading your template—without ever touching LaTeX code. This approach makes resume creation and reformatting fast, accessible, and hassle-free for everyone.

## Future Enhancements

While this notebook streamlines resume creation and LaTeX formatting for applicants, several enhancements could further improve usability, flexibility, and the power of the workflow:

* **Template Marketplace & Customization:**  
  Allow users to browse, preview, and select from a library of LaTeX templates.
* **Support for Additional Input Formats:**  
  Enable importing resumes from DOCX, PDF, or LinkedIn profiles, not just plain text, for broader accessibility.
* **Multilingual Support and Translation:**  
  Add AI-powered translation to create resumes in multiple languages for international applications.
